# DATA PREPROCESSING

##Importing packages and establishing connection to Spotify API

In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 9.2 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

Need a Spotify Account to create app to connect to the API. This account is created for the purposes of the project so none of use need to us our personal accounts.

Email: analyticsproject.g20@gmail.com

Password: MIE3682024

##Getting Spotify API Access Token

In [ ]:
client_id = 'ebc621c56015413ea7055a5721fb7906'
client_secret = '5f7a0f1b1184476da7d14a0a66be199c'

import requests
import base64

def get_access_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = {
        'Authorization': f'Basic {base64.b64encode((client_id + ":" + client_secret).encode()).decode()}'
    }
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

    auth_response = requests.post(auth_url, data=auth_data, headers=auth_header)
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    return access_token

access_token = get_access_token(client_id, client_secret)
print(access_token)

BQALAHUlYkLPOYli7u6YPoJEEoyqCBf8WmnyUYgkPgx7JzLjq7B6Y2KYAgUTswXVDgmzQ1DlyzP6evZ1KRzSa3g6Q88PfENAmkrEd3wuOXks6SALy8g


## Fetching Track information

Getting Artist ID based of their name

In [ ]:
def get_artist_id(name):
    search_url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'q': name,
        'type': 'artist',
        'limit': 1
    }
    response = requests.get(search_url, headers=headers, params=params)
    data = response.json()
    if data['artists']['items']:
        return data['artists']['items'][0]['id']
    else:
        return None

get_artist_id('The Beatles')

'3WrFJ7ztbogyGnTHbHJFl2'

Getting Artist names of most followed Artists on Spotify

In [ ]:
TopArtists = pd.read_csv('Artists.csv')
TopArtistnames = TopArtists['Artist']

Getting top tracks of a given artist

Fetching track information of top tracks

In [ ]:
import time
import requests
import pandas as pd

# Initialize TrackInformation as an empty DataFrame globally
TrackInformation = pd.DataFrame()

def get_artist_top_tracks_with_audio_features(access_token, artist_id, market='US'):
    global TrackInformation  # Declare TrackInformation as global to modify it

    # Endpoint to get artist's top tracks
    top_tracks_url = f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'market': market
    }

    # Request artist's top 10 tracks
    response = requests.get(top_tracks_url, headers=headers, params=params)
    if response.status_code != 200:
        print("Error fetching top tracks:", response.json())
        return []

    data = response.json()

    # Extract track IDs and basic track info
    track_ids = []
    top_tracks = []

    for track in data['tracks']:
        track_id = track['id']
        track_ids.append(track_id)

        # Append track info without release date
        track_info = {
            'track_id': track_id,
            'track_name': track['name'],
            'artist_name': [artist['name'] for artist in track['artists']],
            'popularity': track['popularity'],
            'album': track['album']['name'],
            'uri': track['uri']
        }
        top_tracks.append(track_info)

    # Request audio features for all tracks in one call
    audio_features_url = 'https://api.spotify.com/v1/audio-features'
    audio_features_params = {'ids': ','.join(track_ids)}
    audio_features_response = requests.get(audio_features_url, headers=headers, params=audio_features_params)
    if audio_features_response.status_code != 200:
        print("Error fetching audio features:", audio_features_response.json())
        return []

    audio_features_data = audio_features_response.json()

    # Ensure 'audio_features' key is present in the response
    if 'audio_features' not in audio_features_data:
        print("Error: 'audio_features' not found in response. Full response:", audio_features_data)
        return []

    # Combine track info with corresponding audio features
    complete_track_info = []
    for track_info, audio_features in zip(top_tracks, audio_features_data['audio_features']):
        if audio_features:  # Check if audio features are available
            combined_data = {**track_info, **audio_features}
            complete_track_info.append(combined_data)

    # Append the new data to TrackInformation
    TrackInformation = pd.concat([TrackInformation, pd.DataFrame(complete_track_info)],axis = 0)
    time.sleep(3)


Saving output to a csv file

In [ ]:
def df_to_csv(df):
  df.to_csv('TrackInformation_complete.csv', index=False)

Running functions

In [ ]:
# recording track information of 10 artists at a time (can't do more or else it
# will exceed spotify API's rate limits)

for i in range(0, 2500, 10):
    Batch = TopArtistnames[i:i+10]
    Batch = Batch.tolist()
    for j in range(len(Batch)):
      artist_id = get_artist_id(Batch[j])
      get_artist_top_tracks_with_audio_features(access_token, artist_id)
    print("Batch complete")
    df_to_csv(TrackInformation)


Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete
Batch complete


## Creating final dataset

The API rate limits were exceed multiple times when running the above code, leaving incomplete datasets. Therefore to get the entire track dataset, these incomplete datasets were merged.

In [ ]:
# merging incomplete datasets

file_paths = ["/content/TrackInformation_complete (1).csv", "/content/TrackInformation_complete (2).csv", "/content/TrackInformation_complete (3).csv"]
song_info_df = pd.concat([pd.read_csv(file) for file in file_paths], axis = 0)
song_info_df.shape


(25021, 23)

In [ ]:
# removing duplicates
song_info_df_cleaned = song_info_df.drop_duplicates()
song_info_df_cleaned.shape

(22081, 23)

In [ ]:
# exporting df to csv
song_info_df_cleaned.to_csv('Song_Info.csv', index=False)